<a href="https://colab.research.google.com/github/cij1012/pytorch/blob/master/%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_mydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random

In [64]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("학습", device)

학습 cuda


In [0]:
torch.manual_seed(1012)
if device == 'cuda':
  torch.cuda.manual_seed_all(1012)

In [0]:
kmnist_train = dsets.KMNIST(root='KMNIST_data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
kmnist_test = dsets.KMNIST(root='KMNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

In [0]:
batch_size = 100

data_loader = DataLoader(dataset=kmnist_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)
val_loader = DataLoader(dataset=kmnist_test,
                        batch_size=len(kmnist_test))

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.model = nn.Sequential(
        nn.Linear(784,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,10)
    )
  def forward(self,x):
    x = x.view(-1,28*28)
    return self.model(x)

In [0]:
model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.05)

epochs = 10

In [81]:
total_batch = len(data_loader)+len(val_loader)

train_losses = []
val_losses = []

for epoch in range(epochs):
  avg_cost=0
  for i,data in enumerate(data_loader):
    imgs, labels=data
    imgs = imgs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    hypothesis=model(imgs)
    cost = criterion(hypothesis,labels)
    cost.backward()
    optimizer.step()
    train_losses.append(cost.item())

    avg_cost += cost/total_batch

  print('epoch {} train_cost = {}'.format(epoch+1, avg_cost))

  for i,data in enumerate(val_loader):
    imgs,labels=data
    imgs=imgs.to(device)
    labels=labels.to(device)

    optimizer.zero_grad()
    hypothesis=model(imgs)
    cost=criterion(hypothesis,labels)
    cost.backward()
    optimizer.step()
    val_losses.append(cost.item())

    avg_cost += cost/total_batch
  print('epoch {} val_cost = {}'.format(epoch+1,avg_cost))
  print('-'*30)

epoch 1 train_cost = 0.11841520667076111
epoch 1 val_cost = 0.11923662573099136
------------------------------


KeyboardInterrupt: ignored